<p style="font-size:32px; font-weight: bolder; text-align: center">Equivariant ML models</p>
<p style="font-size:16px; text-align: center">A tutorial introduction</p>
<p style="text-align: center"><i> authored by: <a href="mailto:michele.ceriotti@gmail.com"> Michele Ceriotti </a></i></p>

_Equivariance_ indicates the property of a function for which the inputs and outputs are subject to the action of the same symmetries, and which commutes with the application of the symmetries, that is: $f(\hat{S}A) = \hat{S} f(A)$. _Invariance_ can be seen as a special case, in which  $f(\hat{S}A) = f(A)$.

This notebook provides a brief introduction to the concept of equivariance in the context of atomic-scale machine-learning models, focusing in particular on the case of 3D rotations and inversion - in technical terms the $O(3)$ group symmetries - and their combination with translations - the three-dimensional Euclidean group $E(3)$. 

# The tools of the trade

We will use a number of packages, including both general-purpose ones and some that are designed specifically to the construction and manipulation of equivariant ML models. You should be able to fetch most of these from PIP, while others might require a custom installation.

Besides some time-tested classics...

In [1]:
import numpy as np
import matplotlib.pyplot as plt

... we will use [ASE](https://wiki.fysik.dtu.dk/ase/) to load and manipulate atomic structures 

In [2]:
import ase.io

... [chemiscope](https://chemiscope.org) to visualize them, [equistore](https://github.com/lab-cosmo/equistore) to hold equivariant quantities and [rascaline](https://github.com/Luthaf/rascaline) to compute atom-centered density correlations (ACDCs) - the main family of equivariant descriptors we will use.

In [3]:
import equistore
import chemiscope
from rascaline import SphericalExpansion

## Water dataset

We use as a demonstrative dataset a simple dataset that contains a water molecule, stretched and bent in different ways. For each configuration, the dataset holds energy, forces and dipole moments, computed using the high-accuracy [Partridge-Schwenke fits](10.1063/1.473987). 

In [35]:
frames = ase.io.read("data/chemrev_nuprime-theta-grid_computed.xyz", ":")

In [38]:
cs = chemiscope.show(frames, 
        settings={
            'map' : {'color' : {'property' : 'energy'} },
            'structure': [{'axes': 'xyz','keepOrientation': True}]
        })
display(cs)

/home/michele/local/lib/python3.8/site-packages/chemiscope/structures.py:305: UserWarning: value '[ 0.439333 -0.486919  0.      ]' of type '<class 'numpy.ndarray'>' for the 'force' property from ASE is not convertible to float or string, this property will be ignored.
  warnings.warn(
/home/michele/local/lib/python3.8/site-packages/chemiscope/structures.py:305: UserWarning: value '[-0.00853353  0.49287236  0.        ]' of type '<class 'numpy.ndarray'>' for the 'dipole' property from ASE is not convertible to float or string, this property will be ignored.
  warnings.warn(


ChemiscopeWidget(value=None, data='{"meta": {"name": " "}, "settings": {"map": {"color": {"property": "energy"…

Processes the properties of the structures (as stored in the `.extxyz` file) as `TensorBlock` objects. We will see later how these can be used. 

In [6]:
energy_values = []
force_values = []
dipole_values = []
for f in frames:
    energy_values.append(f.info['energy'])
    dipole_values.append(f.info['dipole'])
    force_values.append(f.arrays['force'])

In [7]:
energies = equistore.TensorBlock(values=np.asarray(energy_values).reshape(-1,1,1), 
                                 samples=equistore.labels.Labels(names = ["structure"], values=np.arange(len(frames), dtype=np.int32).reshape(-1,1) ),
                                 components=[equistore.labels.Labels.single()],
                                 properties=equistore.labels.Labels.single(),
                                )

energies.add_gradient("positions", data=np.asarray(force_values).reshape(-1,3,1,1),
                      samples=equistore.labels.Labels(names=["sample", "atom"], 
                                                      values=np.asarray(np.vstack([ [[i,0],[i,1],[i,2]] for i in range(len(frames))]),dtype=np.int32) ),
                      components=[equistore.labels.Labels(names=["gradient_direction"], values=np.asarray([[0,1,2]], dtype=np.int32).T),
                                  equistore.labels.Labels.single()
                                 ]
                     )

In [8]:
dipoles = equistore.TensorBlock(values=np.asarray(dipole_values).reshape(-1,3,1), 
                                 samples=equistore.labels.Labels(names = ["structure"], values=np.arange(len(frames), dtype=np.int32).reshape(-1,1) ),
                                 components=[equistore.labels.Labels(names=["direction"], values=np.asarray([[0,1,2]], dtype=np.int32).T) ],
                                 properties=equistore.labels.Labels.single(),
                                )

# Rotations and vectors

Let's now start looking into how a vector such as the dipole moment rotates. The key concept is that a vectorial property such as the atomic positions or the dipole moment transform, under the action of a rotation operation $\hat{R}$ in a way consistent with the application of a rotation matrix $\mathbf{R}$, i.e. if a structure $A$ has atomic positions $\mathbf{r}_i$ and dipole moment $\mathbf{y}_i$ (each of these being a 3-vector corresponding to the Cartesian coordinates $(x,y,z)$) then the rotated structure $\hat{R}A$ has atomic coordinates $\mathbf{R}\mathbf{r}_i$ and dipole moment $\mathbf{R}\mathbf{y}$. 


<img src="figures/rotations.png" width="400"/>

A rotation can be defined in terms of [Euler angles](https://en.wikipedia.org/wiki/Euler_angles), a set of three angles $(\alpha, \beta, \gamma)$ that define the orientation of a rigid body relative to a reference frame.  This is a problem that is made quite tricky by the existence of dozen of alternative conventions. For those in the know, we use the $ZYZ$ intrinsic rotations definition, which is also the one commonly used to define quantities in angular momentum theory. We use a wrapper from some utilities that returns the rotation matrix in the "correct" format, and use to generate a set of water molecules in which the position of the atoms and the molecular dipoles have been rotated appropriately. 

In [69]:
from utils.rotations import rotation_matrix

In [72]:
rotated_structures = []
selected_frame = frames[47]
selected_dipole = dipoles.values[94,:,0]
# adds two fake atoms to show the magnitude and direction of the dipole moment
dipole_mol = ase.Atoms("FH", positions=[[0,2,0],selected_dipole+[0,2,0]])
for alpha in np.linspace(0, 2*np.pi, 8):
    for beta in np.linspace(0, np.pi, 4):
        for gamma in np.linspace(0, 2*np.pi, 8):
            rot_frame = selected_frame.copy() + dipole_mol
            rot_frame.info['alpha'] = alpha
            rot_frame.info['beta'] = beta
            rot_frame.info['gamma'] = gamma
            # rotates the frame
            R = rotation_matrix(alpha, beta, gamma)
            rot_frame.positions = rot_frame.positions@R.T
            rot_frame.cell = rot_frame.cell@R.T
            rot_frame.info['dipole'] = rot_frame.info['dipole']@R.T
            rotated_structures.append(rot_frame)

In [73]:
cs = chemiscope.show(rotated_structures, 
        settings={
            'map' : {
                'x' : {'property' : 'alpha'},
                'y' : {'property' : 'beta'},
                'z' : {'property' : 'gamma'},
                'color' : {'property' : 'energy'} },
            'structure': [{'axes': 'xyz','keepOrientation': True}]
        })
display(cs)

/home/michele/local/lib/python3.8/site-packages/chemiscope/structures.py:305: UserWarning: value '[ 0.0797022  -0.00155446  0.        ]' of type '<class 'numpy.ndarray'>' for the 'force' property from ASE is not convertible to float or string, this property will be ignored.
  warnings.warn(
/home/michele/local/lib/python3.8/site-packages/chemiscope/structures.py:305: UserWarning: value '[0.0091859  0.42833807 0.        ]' of type '<class 'numpy.ndarray'>' for the 'dipole' property from ASE is not convertible to float or string, this property will be ignored.
  warnings.warn(


ChemiscopeWidget(value=None, data='{"meta": {"name": " "}, "settings": {"map": {"x": {"property": "alpha"}, "y…

# Rotations and arbitrary tensors

# New heading

In [13]:
a
equistore.TensorMap()

NameError: name 'a' is not defined

In [56]:
hypers = {
    "cutoff": 3.5,
    "max_radial": 6,
    "max_angular": 6,
    "atomic_gaussian_width": 0.3,
    "radial_basis": {"Gto": {}},
    "cutoff_function": {"ShiftedCosine": {"width": 0.5}},
    "center_atom_weight": 1.0,
    "gradients": True,
}

calculator = SphericalExpansion(**hypers)

descriptor = calculator.compute(frames)

In [57]:
frames[0].positions-=50
frames[0].cell *= 0
frames[0].pbc = False

In [58]:
descriptor.block(0).gradient("positions").components

[Labels([(0,), (1,), (2,)], dtype=[('gradient_direction', '<i4')]),
 Labels([(0,)], dtype=[('spherical_harmonics_m', '<i4')])]